# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')

from nltk.corpus import stopwords



[nltk_data] Downloading package punkt to /Users/dikshant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dikshant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dikshant/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [30]:
# import librarires

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import TruncatedSVD
import pickle



In [3]:
# load data from database
engine = create_engine('sqlite:///ProcessedData.db')
df = pd.read_sql_query('select * from CleanData', engine)

In [3]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
# Divide into response
X = df['message']
y = df.drop(['id', 'original', 'message','genre'], axis = 1)

In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """
    Tokenizes text data
    
    Args:
        text: Messages as input
        
    Output:
        word list: list of words which are normalized, and in their root form.
    """
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Substitute everything that is not alphabets or numbers with space
    
    # tokenize text
    words = word_tokenize(text)
    
    # remove stop words
    stopWords = stopwords.words("english") # What does this do
    words = [word for word in words if word not in stopWords]
    
    # extract root form of words
    words = [WordNetLemmatizer().lemmatize(word, pos = 'v') for word in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
                      ('vect', CountVectorizer(tokenizer = tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))
    
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# Split the data into train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1023)

In [9]:
# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
def class_report(classifier, X_test, y_test):
    """
    Generates classification report
    
    Args:
        classifier: model to be used
        X_test: predictor variable of the test dataset
        y_test: response variable of the test dataset
    
    Output:
        Prints the classification report for each categories
    """
    
    # Predict on test data
    y_pred = classifier.predict(X_test)
    
    # print classification report
    for i, column_name in enumerate(y_test):
        print(column_name)
        print(classification_report(y_test[column_name], y_pred[:, i])) # print all rows and ith column
        

In [16]:
# Print Classification report
class_report(pipeline, X_test, y_test) 

related
             precision    recall  f1-score   support

          0       0.62      0.46      0.53      1205
          1       0.85      0.91      0.88      4000
          2       0.52      0.41      0.46        39

avg / total       0.79      0.81      0.79      5244

request
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      4320
          1       0.77      0.42      0.54       924

avg / total       0.87      0.88      0.86      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3064
          1       0.75      0.61      0.67      2180

avg / total       0.75      0.75      0.75      5244

medical_help
             precision    recall  f1-sco

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
cv.fit(X_train, y_train)

# Generate classification report
class_report(cv, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.71      0.44      0.54      1205
          1       0.84      0.94      0.89      4000
          2       0.57      0.44      0.49        39

avg / total       0.81      0.82      0.81      5244

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      4320
          1       0.83      0.49      0.62       924

avg / total       0.89      0.89      0.88      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.79      0.85      0.82      3064
          1       0.76      0.68      0.72      2180

avg / total       0.78      0.78      0.78      5244

medical_help
             precision    recall  f1-sco

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
#Improve  the pipeline
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])


In [24]:
# Train & predict with new pipeline
pipeline2.fit(X_train, y_train)
class_report(pipeline2, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.67      0.00      0.01      1205
          1       0.76      1.00      0.87      4000
          2       1.00      0.05      0.10        39

avg / total       0.74      0.76      0.66      5244

request
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      4320
          1       0.00      0.00      0.00       924

avg / total       0.68      0.82      0.74      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.58      1.00      0.74      3064
          1       0.00      0.00      0.00      2180

avg / total       0.34      0.58      0.43      5244

medical_help
             precision    recall  f1-sco

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
# Tune Parameters
parameters2 = { 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] 
}

In [26]:
# Apply Grid Search
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)


In [27]:
# Fit the grid search
cv2.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__learning_rate': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
# Get classification report for the new grid search 
class_report(cv2, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.67      0.00      0.01      1205
          1       0.76      1.00      0.87      4000
          2       0.00      0.00      0.00        39

avg / total       0.74      0.76      0.66      5244

request
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      4320
          1       0.00      0.00      0.00       924

avg / total       0.68      0.82      0.74      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.58      1.00      0.74      3064
          1       0.00      0.00      0.00      2180

avg / total       0.34      0.58      0.43      5244

medical_help
             precision    recall  f1-sco

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [31]:
# Export to pickle file
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.